In [ ]:
require 'watir'
require 'nokogiri'
require 'dotenv/load'

root_path = ENV['URL'] + 'categorias/'

In [ ]:
# browser = Watir::Browser.new :chrome, headless: true #headless chrome
browser = Watir::Browser.new :chrome

'Start browser'

In [ ]:
browser.goto root_path

In [ ]:
document = Nokogiri::HTML(browser.html)

In [ ]:
categories = document.css("a[class='tags']")

In [ ]:
url = ENV['URL']

paths = categories.map { |c| 
    path = c.attr('href')
  }

In [ ]:
browser.goto url + paths[0]

In [ ]:
pages = Nokogiri::HTML(browser.html)

In [ ]:
last_page = pages.css('.row a').map { |a| 
    a.text
  }.reject { |a| 
    a.include? '>' or a.include? '<' 
  }.last

In [ ]:
recipies = (1..20).map { |index| #just to do a small fetch
#recipies = (1..last_page.to_i).map { |index| # run over all pages, commonly around 2k

    puts index
    browser.goto url + paths[0] + "?page=#{index}"
    recipies = browser.elements(class: ['box-hover', 'box-big-item']).map { |e|
      e.href
    }.map { |r_url| 
      browser.goto r_url
      recipie = Nokogiri::HTML(browser.html)
      ingredients = recipie.at_css("div[id='info-user']").css('li').map { |l| 
        l.text
      }
      recipe_name = recipie.at_css("div[class='recipe-title']").css('h1').text.delete!("\n")
      { recipe_name => ingredients }
    }
    recipies
  }
